# Datasets second level

In [1]:
import numpy as np
import pandas as pd


from pathlib import Path

In [2]:
INPUT_PATH = Path(".").resolve().parent / "input"

train_df = pd.read_csv(INPUT_PATH / "train.csv")

In [7]:
OUTPUT_PATH = Path("../output/").resolve()

fold_pred_paths = [
    OUTPUT_PATH / "2" / "4a3e9debb9de4ac1a36e9aa65d7fbfa8" / "artifacts" / "probas_HPASENet50.csv",
    OUTPUT_PATH / "2" / "2ee3ec80e8aa4dc9a6805cec9e6a49cd" / "artifacts" / "probas_HPASENet50.csv",
    OUTPUT_PATH / "2" / "4940c67fceb8474fbfd835b7e1ec35a4" / "artifacts" / "probas_HPASENet50.csv",
]

fold_preds = [pd.read_csv(p) for p in fold_pred_paths]

In [8]:
fold_preds = pd.concat(fold_preds, axis=0)

In [9]:
fold_preds.shape, train_df.shape

((31072, 29), (31072, 2))

In [10]:
fold_preds.head()

,Id,Nucleoplasm,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,Peroxisomes,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods and rings
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,0.230910,0.006755,0.010593,0.001914,0.000398,0.039371,0.007357,0.354403,0.000323,...,0.718186,0.494725,0.000494,0.005965,0.000795,0.000905,0.005176,0.098934,0.002775,0.000371
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,0.773012,0.972015,0.163404,0.054010,0.023306,0.030100,0.016399,0.771139,0.009911,...,0.021943,0.008368,0.011160,0.004911,0.004071,0.011461,0.005762,0.018520,0.004472,0.001569
2,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,0.008016,0.000125,0.003219,0.004950,0.000051,0.008688,0.000154,0.049075,0.000215,...,0.944682,0.995627,0.001569,0.000512,0.006208,0.000091,0.124953,0.001047,0.007597,0.000140
3,0020af02-bbba-11e8-b2ba-ac1f6b6435d0,0.011767,0.014063,0.991021,0.051161,0.010249,0.005056,0.181499,0.006192,0.011010,...,0.091716,0.082285,0.002290,0.178437,0.030976,0.006063,0.042948,0.722909,0.006090,0.000678
4,00285ce4-bba0-11e8-b2b9-ac1f6b6435d0,0.733706,0.011369,0.943207,0.105690,0.029433,0.013763,0.003800,0.075454,0.004593,...,0.040836,0.072134,0.008395,0.020752,0.040282,0.016119,0.030982,0.044822,0.016995,0.001704


In [11]:
train_df.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [12]:
def get_second_level_train_dataset(train_df, fold_preds, num_tags=28):
    
    image_ids = list(train_df['Id'])    
    targets = np.zeros((len(image_ids), num_tags), dtype=np.uint8)
    for i, tags in enumerate(train_df['Target']):
        tags = tags.split(" ")
        for t in tags:
            targets[i, int(t)] = 1
    
    out = fold_preds.copy()
    out.index = out['Id']
    out = out.loc[train_df['Id'], :]
    out.index = list(range(len(image_ids)))
    return out, targets

In [13]:
x, y = get_second_level_train_dataset(train_df, fold_preds)

In [14]:
x.head()

,Id,Nucleoplasm,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,Peroxisomes,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods and rings
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,0.230910,0.006755,0.010593,0.001914,0.000398,0.039371,0.007357,0.354403,0.000323,...,0.718186,0.494725,0.000494,0.005965,0.000795,0.000905,0.005176,0.098934,0.002775,0.000371
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,0.773012,0.972015,0.163404,0.054010,0.023306,0.030100,0.016399,0.771139,0.009911,...,0.021943,0.008368,0.011160,0.004911,0.004071,0.011461,0.005762,0.018520,0.004472,0.001569
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,0.200243,0.015802,0.021579,0.088383,0.009820,0.897265,0.009949,0.011853,0.003551,...,0.026200,0.042325,0.011128,0.012238,0.011059,0.050447,0.040150,0.035353,0.024083,0.002142
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,0.464482,0.999715,0.056282,0.001041,0.005443,0.001374,0.135459,0.024367,0.001138,...,0.001201,0.000795,0.000767,0.007798,0.000493,0.024727,0.000334,0.057544,0.000478,0.000129
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,0.008016,0.000125,0.003219,0.004950,0.000051,0.008688,0.000154,0.049075,0.000215,...,0.944682,0.995627,0.001569,0.000512,0.006208,0.000091,0.124953,0.001047,0.007597,0.000140


In [15]:
cols = list(x.columns)
cols.remove("Id")

In [47]:
from sklearn.model_selection import cross_validate 

from sklearn.metrics import f1_score, recall_score, precision_score

In [38]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

fold_index = 0
random_state = 7
n_splits = 5

skf = MultilabelStratifiedKFold(n_splits=n_splits, random_state=random_state)
train_fold_indices = None
val_fold_indices = None
for i, (train_indices, val_indices) in enumerate(skf.split(y, y)):
    if i == fold_index:
        train_fold_indices = train_indices
        val_fold_indices = val_indices
        break

In [40]:
x_trainval = x[cols].values[train_indices, :]
y_trainval = y[train_indices, :]

x_test = x[cols].values[val_indices, :]
y_test = y[val_indices, :]

In [41]:
x_trainval.shape, y_trainval.shape

((24872, 28), (24872, 28))

In [42]:
x_test.shape, y_test.shape

((6200, 28), (6200, 28))

Baseline 

In [117]:
np.eye(1, 28)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [120]:
default_value = np.zeros((28), dtype=np.uint8)
default_value[0] = 1

res = (np.round(x_trainval)).astype(np.uint8)
res[res.sum(axis=1) == 0] = default_value 

In [123]:
class BaselineEstimator:
    
    def fit(self, x, y):
        pass

    def predict(self, x):
        default_value = np.zeros((28), dtype=np.uint8)
        default_value[0] = 1        
        res = (np.round(x)).astype(np.uint8)
        res[res.sum(axis=1) == 0] = default_value 
        return res

In [124]:
estimator = BaselineEstimator()

In [125]:
y_trainval_preds = estimator.predict(x_trainval)
y_test_preds = estimator.predict(x_test)


f1_score(y_test, y_test_preds, average='samples'), f1_score(y_trainval, y_trainval_preds, average='samples')

(0.6781278801843318, 0.6761276784757004)

Learned thresholds baseline

In [126]:
x = x_trainval
y = y_trainval


n_tags = 28
best_thresholds = np.array([0.3 for i in range(n_tags)])

default_value = np.zeros((n_tags), dtype=np.uint8)
default_value[0] = 1        


sorted_indices = np.argsort(y.sum(axis=0))[::-1]
best_score = 0.0
thresholds = np.arange(0.9, 0.1, -0.01)
for _ in range(3):
    sorted_indices = sorted_indices[::-1]
    print("\n-- Try ", _) 
    for idx in sorted_indices:
        print("test index: ", idx)
        for t in thresholds:        
            tt = best_thresholds.copy()
            tt[idx] = t
            y_preds = (x > tt).astype(np.uint8)
            y_preds[y_preds.sum(axis=1) == 0] = default_value 
            score = f1_score(y, y_preds, average='samples')
            if score > best_score:
                best_thresholds[idx] = t
                best_score = score
                print("best score: ", best_score)            


-- Try  0
test index:  27
best score:  0.6928248505363334
test index:  15
test index:  10
test index:  9
test index:  8
test index:  20
best score:  0.6928449534633196
best score:  0.6928554835679314
best score:  0.6928648649338582
best score:  0.6928997100073009
test index:  17
best score:  0.6929032519515794
best score:  0.6929332148856112
best score:  0.6929479570320677
test index:  24
best score:  0.6929532220843736
test index:  26
best score:  0.6929744418406368
best score:  0.6930012457432849
best score:  0.693007946718947
best score:  0.6931285642808641
best score:  0.6932666043795025
best score:  0.6933470160874471
best score:  0.6935333032108524
best score:  0.6935737005212721
best score:  0.6935884426677287
best score:  0.6936202244380115
test index:  16
best score:  0.693728014417947
best score:  0.6937322264597916
best score:  0.6937429480208509
test index:  13
best score:  0.6937573072544125
best score:  0.6938124467112888
best score:  0.693878116272777
best score:  0.693

In [127]:
best_score, best_thresholds

(0.7027182667320171,
 array([0.41, 0.33, 0.3 , 0.34, 0.35, 0.3 , 0.4 , 0.37, 0.3 , 0.3 , 0.3 ,
        0.45, 0.54, 0.46, 0.38, 0.3 , 0.49, 0.33, 0.36, 0.26, 0.15, 0.28,
        0.46, 0.32, 0.36, 0.41, 0.12, 0.9 ]))

In [111]:
best_score, best_thresholds

(0.700420119264603,
 array([0.4 , 0.33, 0.27, 0.34, 0.35, 0.3 , 0.4 , 0.37, 0.3 , 0.3 , 0.3 ,
        0.45, 0.54, 0.46, 0.38, 0.3 , 0.49, 0.33, 0.29, 0.26, 0.15, 0.27,
        0.46, 0.32, 0.36, 0.4 , 0.11, 0.9 ]))

In [106]:
best_score, best_thresholds

(0.7003373143510647,
 array([0.4 , 0.34, 0.27, 0.34, 0.35, 0.3 , 0.4 , 0.37, 0.25, 0.25, 0.25,
        0.45, 0.54, 0.46, 0.39, 0.25, 0.49, 0.33, 0.29, 0.26, 0.15, 0.28,
        0.38, 0.32, 0.36, 0.4 , 0.11, 0.25]))

In [132]:
from sklearn.model_selection import GridSearchCV


class LearnedThresholdsBaselineEstimator:
    
    def __init__(self, n_tags):
        self.n_tags = n_tags
        self.best_thresholds = np.array([0.3 for i in range(n_tags)])
        self.default_value = np.zeros((n_tags), dtype=np.uint8)
        self.default_value[0] = 1        

    def fit(self, x, y):        
        sorted_indices = np.argsort(y.sum(axis=0))[::-1]
        best_score = 0.0
        thresholds = np.arange(0.9, 0.1, -0.01)
        for r in range(3):
            sorted_indices = sorted_indices[::-1]
            print("\n-- Try ", r) 
            for idx in sorted_indices:
                print("test index: ", idx)
                for t in thresholds:        
                    tt = self.best_thresholds.copy()
                    tt[idx] = t
                    y_preds = (x > tt).astype(np.uint8)
                    y_preds[y_preds.sum(axis=1) == 0] = self.default_value 
                    score = f1_score(y, y_preds, average='samples')
                    if score > best_score:
                        self.best_thresholds[idx] = t
                        best_score = score
        print("best score: ", best_score)

    def predict(self, x):
        y_preds = (x > self.best_thresholds).astype(np.uint8)
        y_preds[y_preds.sum(axis=1) == 0] = self.default_value 
        return y_preds
   

In [133]:
estimator = LearnedThresholdsBaselineEstimator(n_tags=28)

estimator.fit(x_trainval, y_trainval)


-- Try  0
test index:  27
test index:  15
test index:  10
test index:  9
test index:  8
test index:  20
test index:  17
test index:  24
test index:  26
test index:  16
test index:  13
test index:  12
test index:  22
test index:  18
test index:  6
test index:  14
test index:  11
test index:  1
test index:  19
test index:  3
test index:  4
test index:  5
test index:  7
test index:  23
test index:  2
test index:  21
test index:  25
test index:  0

-- Try  1
test index:  0
test index:  25
test index:  21
test index:  2
test index:  23
test index:  7
test index:  5
test index:  4
test index:  3
test index:  19
test index:  1
test index:  11
test index:  14
test index:  6
test index:  18
test index:  22
test index:  12
test index:  13
test index:  16
test index:  26
test index:  24
test index:  17
test index:  20
test index:  8
test index:  9
test index:  10
test index:  15
test index:  27

-- Try  2
test index:  27
test index:  15
test index:  10
test index:  9
test index:  8
test index:  

In [134]:
y_trainval_preds = estimator.predict(x_trainval)
y_test_preds = estimator.predict(x_test)

f1_score(y_test, y_test_preds, average='samples'), f1_score(y_trainval, y_trainval_preds, average='samples')

(0.7007465437788019, 0.7027182667320171)

In [154]:
estimator.best_thresholds

array([0.41, 0.33, 0.3 , 0.34, 0.35, 0.3 , 0.4 , 0.37, 0.3 , 0.3 , 0.3 ,
       0.45, 0.54, 0.46, 0.38, 0.3 , 0.49, 0.33, 0.36, 0.26, 0.15, 0.28,
       0.46, 0.32, 0.36, 0.41, 0.12, 0.9 ])

GBM Improved baseline

In [142]:
n_estimators = 28
params = {
    'learning_rate': 0.01,
    'num_leaves':31,
    'objective':'binary'
}
models = []


for i in range(n_estimators):      
    print("Train {} estimator".format(i))
    train_data = lgb.Dataset(x, label=y[:, i])            
    result = lgb.cv(params, train_data, 
                    nfold=3, stratified=True,
                    num_boost_round=200,
                    metrics=['auc', ], 
                    seed=12,
                    verbose_eval=1,
                    early_stopping_rounds=10)
    num_boost_round=np.argmax(result['auc-mean'])
    num_boost_round = 1 if num_boost_round == 0 else num_boost_round
    models.append(lgb.train(params, train_data, num_boost_round=num_boost_round))


Train 0 estimator
[1]	cv_agg's auc: 0.929387 + 0.00138155
[2]	cv_agg's auc: 0.929884 + 0.00126364
[3]	cv_agg's auc: 0.930307 + 0.00168621
[4]	cv_agg's auc: 0.930389 + 0.00162505
[5]	cv_agg's auc: 0.930812 + 0.00127783
[6]	cv_agg's auc: 0.930863 + 0.00126171
[7]	cv_agg's auc: 0.931265 + 0.00140797
[8]	cv_agg's auc: 0.931608 + 0.00174638
[9]	cv_agg's auc: 0.931783 + 0.00186158
[10]	cv_agg's auc: 0.931799 + 0.00186139
[11]	cv_agg's auc: 0.931921 + 0.00174623
[12]	cv_agg's auc: 0.932005 + 0.00169954
[13]	cv_agg's auc: 0.932218 + 0.00171435
[14]	cv_agg's auc: 0.932281 + 0.00179161
[15]	cv_agg's auc: 0.932329 + 0.00183425
[16]	cv_agg's auc: 0.93253 + 0.00192624
[17]	cv_agg's auc: 0.932573 + 0.00195155
[18]	cv_agg's auc: 0.932642 + 0.0020335
[19]	cv_agg's auc: 0.932638 + 0.00208496
[20]	cv_agg's auc: 0.93282 + 0.00216037
[21]	cv_agg's auc: 0.93287 + 0.00219115
[22]	cv_agg's auc: 0.932962 + 0.0022194
[23]	cv_agg's auc: 0.933079 + 0.00227229
[24]	cv_agg's auc: 0.93315 + 0.00227662
[25]	cv_agg's

In [146]:

y_preds = np.zeros_like(x_test)
for i in range(n_estimators):     
    test_data = lgb.Dataset(x_test)            
    y_preds[:, i] = models[i].predict(x_test)

In [152]:
y_preds = (np.round(y_preds)).astype(np.uint8)
y_preds[y_preds.sum(axis=1) == 0] = default_value 

In [153]:
f1_score(y_test, y_preds, average='samples')

0.6780199692780339

In [75]:
import lightgbm as lgb


class ImprovedBaselineEstimator:
    
    def __init__(self, n_tags):
        self.n_estimators = n_tags
        self.params = {
            'learning_rate': 0.01,
            'num_leaves':31,
            'objective':'binary'
        }
        self.models = []
    
    def fit(self, x, y):
        for i in range(self.n_estimators):            
            print("Train {} estimator".format(i))
            train_data = lgb.Dataset(x, label=y[:, i])            
            result = lgb.cv(self.params, train_data, 
                            nfold=3, stratified=True,
                            num_boost_round=200,
                            metrics=['auc', ], 
                            seed=12,
                            verbose_eval=1,
                            early_stopping_rounds=10)

            self.models.append(lgb.train(self.params, train_data, num_round=result.best_iteration))

    def predict(self, x):        
        y_probas = np.zeros((len(x), self.n_estimators), dtype=np.float32)        
        for i, model in enumerate(self.models):
            y_probas[:, i] = model.predict(x)        
        return (np.round(y_probas)).astype(np.uint8)

In [76]:
estimator = ImprovedBaselineEstimator(n_tags=28)

estimator.fit(x_trainval, y_trainval)

Train 0 estimator
[1]	cv_agg's auc: 0.929387 + 0.00138155
[2]	cv_agg's auc: 0.929884 + 0.00126364
[3]	cv_agg's auc: 0.930307 + 0.00168621
[4]	cv_agg's auc: 0.930389 + 0.00162505
[5]	cv_agg's auc: 0.930812 + 0.00127783
[6]	cv_agg's auc: 0.930863 + 0.00126171
[7]	cv_agg's auc: 0.931265 + 0.00140797
[8]	cv_agg's auc: 0.931608 + 0.00174638
[9]	cv_agg's auc: 0.931783 + 0.00186158
[10]	cv_agg's auc: 0.931799 + 0.00186139
[11]	cv_agg's auc: 0.931921 + 0.00174623
[12]	cv_agg's auc: 0.932005 + 0.00169954
[13]	cv_agg's auc: 0.932218 + 0.00171435
[14]	cv_agg's auc: 0.932281 + 0.00179161
[15]	cv_agg's auc: 0.932329 + 0.00183425
[16]	cv_agg's auc: 0.93253 + 0.00192624
[17]	cv_agg's auc: 0.932573 + 0.00195155
[18]	cv_agg's auc: 0.932642 + 0.0020335
[19]	cv_agg's auc: 0.932638 + 0.00208496
[20]	cv_agg's auc: 0.93282 + 0.00216037
[21]	cv_agg's auc: 0.93287 + 0.00219115
[22]	cv_agg's auc: 0.932962 + 0.0022194
[23]	cv_agg's auc: 0.933079 + 0.00227229
[24]	cv_agg's auc: 0.93315 + 0.00227662
[25]	cv_agg's

AttributeError: 'dict' object has no attribute 'best_iteration'

In [62]:
y_trainval_preds = estimator.predict(x_trainval)
y_test_preds = estimator.predict(x_test)


f1_score(y_test, y_test_preds, average='samples'), f1_score(y_trainval, y_trainval_preds, average='samples')

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(0.6572342549923196, 0.6890249506042366)

## Ensemble inferences

In [162]:
import os
from pathlib import Path

import pandas as pd

os.environ['MLFLOW_TRACKING_URI'] = "../output"

In [163]:
def predictions_path(client, run_uuid, weights_filename):
    path = Path(client.tracking_uri)
    run_info = client.get_run(run_id=run_uuid)
    artifact_uri = run_info.info.artifact_uri
    artifact_uri = artifact_uri[artifact_uri.find("/") + 1:]
    path /= Path(artifact_uri) / weights_filename
    # path = Path(artifact_uri) / weights_filename
    assert path.exists(), "File is not found at {}".format(path.as_posix())
    return path.as_posix()

In [164]:
inferences = [
    ("54403c290e4b4d549a2d48949d550d63", "preds_HPASENet50.csv"),
    ("3a03ca24c91f4ca5a730ed0138b34c89", "preds_HPASENet50.csv"),
    ("783350427905429c8c822d2a89706d3a", "preds_HPASENet50.csv")
]

In [235]:
import mlflow


client = mlflow.tracking.MlflowClient(tracking_uri=os.environ['MLFLOW_TRACKING_URI'])

fold_pred_paths = [predictions_path(client, run_id, filename) for run_id, filename in inferences]
fold_preds = [pd.read_csv(p) for p in fold_pred_paths]

for p in fold_preds:
    p.index = p['Id']
    p.drop(columns=['Id'], inplace=True)

In [183]:
res = fold_preds[0]

for i in range(1, len(fold_preds)):
    res += fold_preds[i]

res /= len(fold_preds)

In [184]:
res.head()

,Nucleoplasm,Nuclear membrane,Nucleoli,Nucleoli fibrillar center,Nuclear speckles,Nuclear bodies,Endoplasmic reticulum,Golgi apparatus,Peroxisomes,Endosomes,...,Microtubule organizing center,Centrosome,Lipid droplets,Plasma membrane,Cell junctions,Mitochondria,Aggresome,Cytosol,Cytoplasmic bodies,Rods and rings
Id,,,,,,,,,,,,,,,,,,,,,
00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0.169471,0.034272,0.995812,0.014713,0.002748,0.002149,0.010663,0.006510,0.001851,0.000861,...,0.017876,0.017942,0.000864,0.006049,0.003560,0.005100,0.026282,0.016145,0.001257,0.000240
0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0.155642,0.125737,0.049329,0.107822,0.069890,0.409060,0.101705,0.030404,0.010658,0.006740,...,0.058862,0.088054,0.019482,0.039811,0.022393,0.057908,0.030212,0.377194,0.047398,0.004923
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0.853994,0.006704,0.111919,0.226950,0.051340,0.178544,0.050487,0.040751,0.001799,0.001651,...,0.028883,0.043440,0.003213,0.195909,0.015950,0.035855,0.003832,0.614754,0.009562,0.001500
0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0.729113,0.010369,0.067994,0.020772,0.022476,0.076083,0.013922,0.127673,0.001019,0.001087,...,0.069893,0.064043,0.004917,0.118872,0.013838,0.058084,0.006620,0.508885,0.025080,0.002442
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0.362693,0.010696,0.021928,0.009264,0.016589,0.033237,0.071595,0.094788,0.003092,0.001923,...,0.020606,0.013339,0.007385,0.034271,0.006567,0.337648,0.003998,0.423693,0.007347,0.001290


In [223]:
best_thresholds = np.array([0.41, 0.33, 0.3, 0.34, 0.35, 0.3 , 0.4 , 0.37, 0.3, 0.3, 0.3,
                            0.45, 0.54, 0.46, 0.38, 0.3 , 0.49, 0.33, 0.36, 0.26, 0.15, 0.28,
                            0.46, 0.32, 0.36, 0.41, 0.12, 0.9 ])



def get_predictions(y_probas, thresholds):
    y_preds = (y_probas > thresholds).astype(np.uint8)
    default_value = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.uint8)
    y_preds[y_preds.sum(axis=1) == 0] = default_value
    
    data = []
    for y in y_preds:
        res = " ".join([str(v) for v in np.where(y > 0)[0]])
        data.append(res)    
    
    return data

In [189]:
y_probas = res.values

best_thresholds = np.array([0.41, 0.33, 0.3, 0.34, 0.35, 0.3 , 0.4 , 0.37, 0.3, 0.3, 0.3,
                            0.45, 0.54, 0.46, 0.38, 0.3 , 0.49, 0.33, 0.36, 0.26, 0.15, 0.28,
                            0.46, 0.32, 0.36, 0.41, 0.12, 0.9 ])

y_preds = (y_probas > best_thresholds).astype(np.uint8)

In [192]:
default_value = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=np.uint8)

y_preds[y_preds.sum(axis=1) == 0] = default_value 

In [193]:
y_preds

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0]], dtype=uint8)

In [204]:
data = []
for y in y_preds:
    res = " ".join([str(v) for v in np.where(y > 0)[0]])
    data.append(res)

In [209]:
sample_submission_df['Predicted'] = data

In [210]:
sample_submission_df.to_csv("ensemble_senet50.csv", index=None)

In [215]:
sample_submission_df.head(10)

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,23 25
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,4
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,4 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 25


In [236]:
# fold_preds[0].loc[:, 'Predictions'] = get_predictions(fold_preds[0].values, best_thresholds)

fold_preds[0]['Predictions'].head()

Id
00008af0-bad0-11e8-b2b8-ac1f6b6435d0       2
0000a892-bacf-11e8-b2b8-ac1f6b6435d0      25
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0    0 25
0008baca-bad7-11e8-b2b9-ac1f6b6435d0    0 25
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0    0 11
Name: Predictions, dtype: object

In [237]:
# fold_preds[1].loc[:, 'Predictions'] = get_predictions(fold_preds[1].values, best_thresholds)

fold_preds[1]['Predictions'].head()

Id
00008af0-bad0-11e8-b2b8-ac1f6b6435d0         2
0000a892-bacf-11e8-b2b8-ac1f6b6435d0         5
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0    0 3 25
0008baca-bad7-11e8-b2b9-ac1f6b6435d0      0 25
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0     23 25
Name: Predictions, dtype: object

In [238]:
# fold_preds[2].loc[:, 'Predictions'] = get_predictions(fold_preds[2].values, best_thresholds)

fold_preds[2]['Predictions'].head()

Id
00008af0-bad0-11e8-b2b8-ac1f6b6435d0          2
0000a892-bacf-11e8-b2b8-ac1f6b6435d0          5
0006faa6-bac7-11e8-b2b7-ac1f6b6435d0       0 25
0008baca-bad7-11e8-b2b9-ac1f6b6435d0       0 25
000cce7e-bad4-11e8-b2b8-ac1f6b6435d0    0 23 25
Name: Predictions, dtype: object

In [245]:
a = [10, 20, 30]
c1 = 5
c2 = 10
c3 = 15
c4 = 20
c5 = 25


In [251]:
for c in range(50):
    print(c, sum([v <= c for v in a]))

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 1
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 2
21 2
22 2
23 2
24 2
25 2
26 2
27 2
28 2
29 2
30 3
31 3
32 3
33 3
34 3
35 3
36 3
37 3
38 3
39 3
40 3
41 3
42 3
43 3
44 3
45 3
46 3
47 3
48 3
49 3


In [246]:
[v < c1 for v in a], [v < c2 for v in a], [v < c3 for v in a], [v < c4 for v in a], [v < c5 for v in a]


([False, False, False],
 [False, False, False],
 [True, False, False],
 [True, False, False],
 [True, True, False])

In [247]:
sum([v < c1 for v in a]), sum([v < c2 for v in a]), sum([v < c3 for v in a]), sum([v < c4 for v in a]), sum([v < c5 for v in a])


(0, 0, 1, 1, 2)

TypeError: unsupported operand type(s) for -: 'list' and 'int'